<a href="https://colab.research.google.com/github/Lisavetti/DaTaAnalys_1/blob/main/lab3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Load the dataset
file_path = 'fraud_dataset.csv'
df = pd.read_csv(file_path)

# Separate features and target variable
target = 'fraud_label'  # Update this if the target column name is different
X = df.drop(target, axis=1)
y = df[target]

# Identify categorical and numerical columns
categorical_cols = X.select_dtypes(include=['object', 'category']).columns.tolist()
numerical_cols = X.select_dtypes(include=['number']).columns.tolist()

# Create a preprocessing pipeline
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_cols),
        ('cat', OneHotEncoder(), categorical_cols)
    ])

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Apply the preprocessing pipeline
X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test)


In [12]:
from sklearn.utils.class_weight import compute_class_weight

# Calculate class weights
class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
class_weight_dict = dict(enumerate(class_weights))

print("Class weights:", class_weight_dict)


Class weights: {0: 0.6415094339622641, 1: 2.2666666666666666}


In [13]:
import tensorflow as tf

# Define the model
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [14]:
# Train the model
history = model.fit(X_train, y_train, epochs=100, batch_size=32,
                    validation_split=0.2, class_weight=class_weight_dict)


Epoch 1/100
2/2 [==============================] - 1s 219ms/step - loss: 0.5946 - accuracy: 0.5556 - val_loss: 0.6792 - val_accuracy: 0.5714
Epoch 2/100
2/2 [==============================] - 0s 51ms/step - loss: 0.6893 - accuracy: 0.5000 - val_loss: 0.6646 - val_accuracy: 0.5714
Epoch 3/100
2/2 [==============================] - 0s 37ms/step - loss: 0.6456 - accuracy: 0.4815 - val_loss: 0.6511 - val_accuracy: 0.5000
Epoch 4/100
2/2 [==============================] - 0s 39ms/step - loss: 0.6655 - accuracy: 0.5370 - val_loss: 0.6385 - val_accuracy: 0.5000
Epoch 5/100
2/2 [==============================] - 0s 34ms/step - loss: 0.6251 - accuracy: 0.5926 - val_loss: 0.6271 - val_accuracy: 0.5714
Epoch 6/100
2/2 [==============================] - 0s 34ms/step - loss: 0.6059 - accuracy: 0.6111 - val_loss: 0.6170 - val_accuracy: 0.5714
Epoch 7/100
2/2 [==============================] - 0s 36ms/step - loss: 0.6261 - accuracy: 0.5926 - val_loss: 0.6070 - val_accuracy: 0.6429
Epoch 8/100
2/2 [==

In [15]:
# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss}")
print(f"Test Accuracy: {accuracy*100:.2f}%")


1/1 [==============================] - 0s 27ms/step - loss: 0.0297 - accuracy: 1.0000
Test Loss: 0.029707245528697968
Test Accuracy: 100.00%


Цей код створює простий CNN для класифікації рукописних цифр. Спочатку він завантажує і нормалізує набір даних MNIST, потім створює сверточну нейронну сеть, навчає її на дані і оцінює на тестовому наборі даних.

In [16]:
import tensorflow as tf
from tensorflow.keras import layers, models

# Шаг 1: Загрузка данных MNIST
(mnist_train_images, mnist_train_labels), (mnist_test_images, mnist_test_labels) = tf.keras.datasets.mnist.load_data()

# Нормализация данных: Преобразуем значения пикселей из диапазона [0, 255] в [0, 1]
mnist_train_images = mnist_train_images / 255.0
mnist_test_images = mnist_test_images / 255.0

# Решейпинг данных для CNN
mnist_train_images = mnist_train_images.reshape((-1, 28, 28, 1))
mnist_test_images = mnist_test_images.reshape((-1, 28, 28, 1))

# Шаг 2: Создание модели CNN
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(10, activation='softmax')
])

# Шаг 3: Компиляция модели
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Шаг 4: Обучение модели
model.fit(mnist_train_images, mnist_train_labels, epochs=5, batch_size=64)

# Шаг 5: Оценка модели
test_loss, test_acc = model.evaluate(mnist_test_images, mnist_test_labels)
print(f"Точность на тестовых данных: {test_acc:.4f}")


11490434/11490434 [==============================] - 0s 0us/step
Epoch 1/5
938/938 [==============================] - 72s 74ms/step - loss: 0.1912 - accuracy: 0.9417
Epoch 2/5
938/938 [==============================] - 53s 56ms/step - loss: 0.0514 - accuracy: 0.9838
Epoch 3/5
938/938 [==============================] - 52s 55ms/step - loss: 0.0354 - accuracy: 0.9890
Epoch 4/5
938/938 [==============================] - 53s 57ms/step - loss: 0.0287 - accuracy: 0.9909
Epoch 5/5
313/313 [==============================] - 5s 15ms/step - loss: 0.0259 - accuracy: 0.9923
Точность на тестовых данных: 0.9923


In [21]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, Bidirectional, LSTM, Dense, Dropout
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import re

data = pd.read_csv('googleplaystore_user_reviews.csv')
# Data Preprocessing
# Remove NaN values
data.dropna(subset=['Translated_Review', 'Sentiment'], inplace=True)

# Text Cleaning function
def clean_text(text):
    text = text.lower()
    text = re.sub(r'[^a-z0-9\s]', '', text)
    return text

# Apply text cleaning
data['Translated_Review'] = data['Translated_Review'].apply(clean_text)

# Splitting the dataset into features and target
X = data['Translated_Review']
y = data['Sentiment']

# Encode the target variable
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Tokenization and Padding
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X)
X_seq = tokenizer.texts_to_sequences(X)
X_pad = pad_sequences(X_seq, maxlen=100)

# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_pad, y_encoded, test_size=0.2, random_state=42)

# Building the Bidirectional LSTM model
vocab_size = len(tokenizer.word_index) + 1

model = Sequential()
model.add(Embedding(vocab_size, 128, input_length=100))
model.add(Bidirectional(LSTM(64)))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(3, activation='softmax')) # 3 for three sentiment categories

# Compiling the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.summary()



Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 100, 128)          3157376   
                                                                 
 bidirectional_3 (Bidirecti  (None, 128)               98816     
 onal)                                                           
                                                                 
 dense_18 (Dense)            (None, 64)                8256      
                                                                 
 dropout_7 (Dropout)         (None, 64)                0         
                                                                 
 dense_19 (Dense)            (None, 3)                 195       
                                                                 
Total params: 3264643 (12.45 MB)
Trainable params: 3264643 (12.45 MB)
Non-trainable params: 0 (0.00 Byte)
______________